In [1]:
%matplotlib inline

In [2]:
import shutil, errno
from glob import glob as glb

def copyanything(src, dest_dir):
    for filename in glb(os.path.join(src, '*.*')):
        shutil.copy(filename, dest_dir)


In [3]:
def PDF(url):
    return HTML('<iframe src=./tmp_figures/%s width=700 height=350></iframe>' % url)

This notebook is aimed at getting all the effector candidates from the Pst_104E_genome as defined as the following.
EffectorP prediction done by Jana Sperschneider
Gene expression cluster analysis done by Jana Sperschneider picking cluster 2, 3 and 8
Out of this list two BUSCOs were removed as they are pretty conserved proteins and are simply coming from the upregulated cluster.

In [4]:
import os
from Bio import SeqIO
import pandas as pd
import re
from pybedtools import BedTool
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

In [5]:
#define your input folders
#define your input folders updated for haplotigs
CLUSTER_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/gene_expression/Pst104_p_SecretomeClustering'
EFFECTORP_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/Secretomes/EffectorP'
GFF_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
PROTEIN_ANNO_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/pa_26062017'
OUT_FOLDER = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists'
OUT_FOLDER_FIG = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/figures'
TMP_FIG_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/downstream_analysis_2017/scripts/tmp_figures'

In [6]:
if not os.path.exists(OUT_FOLDER_FIG):
    os.mkdir(OUT_FOLDER_FIG)

In [7]:
genome = 'Pst_104E_v12_'
p_effector_list = []
h_effector_list = []
p_effector_seq_list = []
h_effector_seq_list = []
p_effectorp_list = []
h_effectorp_list = []
p_effectorp_seq_list = []
h_effectorp_seq_list = []

In [8]:
#define what you want to take clusters are from the expression analysis
clusters = [ 2,3,8]
clusters_files = [os.path.join(CLUSTER_FOLDER, x) for x in os.listdir(CLUSTER_FOLDER)\
                 if x.startswith('Cluster') and x.endswith('_DEs.fasta') and\
                  any(str(y) in x for y in clusters) ] #fixed to check if any of the clusters are
                                    #in the file header
effectorp_files = [os.path.join(EFFECTORP_FOLDER, x) for x in os.listdir(EFFECTORP_FOLDER)\
                  if x.endswith('effectors.fasta') and x.startswith(genome)]

In [9]:
clusters_files

['/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/gene_expression/Pst104_p_SecretomeClustering/Cluster8_DEs.fasta',
 '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/gene_expression/Pst104_p_SecretomeClustering/Cluster3_DEs.fasta',
 '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Pst_104E_genome/gene_expression/Pst104_p_SecretomeClustering/Cluster2_DEs.fasta']

In [10]:
#get all the sequence names into a list from the fasta headers 
for file in clusters_files:
    fh = open(file, 'r')
    for seq in SeqIO.parse(fh, 'fasta'):
        if 'hcontig' in seq.id:
            h_effector_list.append(seq.id)
            h_effector_seq_list.append(seq)
        if 'pcontig' in seq.id:
            p_effector_list.append(seq.id)
            p_effector_seq_list.append(seq)
    fh.close()

for file in effectorp_files:
    fh = open(file, 'r')
    for seq in SeqIO.parse(fh, 'fasta'):
        if 'hcontig' in seq.id and seq.id not in h_effector_list:
            h_effector_list.append(seq.id)
            h_effector_seq_list.append(seq)
        if 'pcontig' in seq.id and seq.id not in p_effector_list:
            p_effector_list.append(seq.id)
            p_effector_seq_list.append(seq)
    fh.close()
    
for file in effectorp_files:
    fh = open(file, 'r')
    for seq in SeqIO.parse(fh, 'fasta'):
        if 'hcontig' in seq.id:
            h_effectorp_list.append(seq.id)
            h_effectorp_seq_list.append(seq)
        if 'pcontig' in seq.id:
            p_effectorp_list.append(seq.id)
            p_effectorp_seq_list.append(seq)
    fh.close()

In [11]:
#define the effector file name
p_effector_file = os.path.join(OUT_FOLDER, genome + 'p_effector.list')
p_effectorp_file = os.path.join(OUT_FOLDER, genome + 'p_effectorp.list')

In [12]:
#now get BUSCO list in order to remove the two BUSCOs from the effector candidate list
p_busco_file = [os.path.join(PROTEIN_ANNO_FOLDER, x) for x in os.listdir(PROTEIN_ANNO_FOLDER) if x.startswith(genome+'p_ctg') and 'busco' in x][0]
p_busco_list = pd.read_csv(p_busco_file, header=None, sep='\t')[0].tolist()

In [13]:
#write out effectors without BUSCOs
effector_busco_overlap = [x for x in p_effector_list if x in p_busco_list]
print(effector_busco_overlap)
#remove those two from the effector list and update the effectors
#one is a peptidase and the other an ER cargo protein both are fairly conserved
print("This is the number of effector candidates before removal: %i" % len(p_effector_list))
updated_effector_seq_list = []
for x in effector_busco_overlap:
    p_effector_list.remove(x)
for seq in p_effector_seq_list:
    if seq.id not in effector_busco_overlap:
        updated_effector_seq_list.append(seq)
print("This is the number of effector candidates after removal of BUSCOs: %i" % len(p_effector_list))

['evm.model.pcontig_041.167', 'evm.model.pcontig_042.2']
This is the number of effector candidates before removal: 1574
This is the number of effector candidates after removal of BUSCOs: 1572


In [14]:
#write out effectorps without BUSCOs
effectorp_busco_overlap = [x for x in p_effectorp_list if x in p_busco_list]
print(effectorp_busco_overlap)
#remove those two from the effector list and update the effectors
#one is a peptidase and the other an ER cargo protein both are fairly conserved
print("This is the number of effector candidates before removal: %i" % len(p_effectorp_list))
updated_effectorp_seq_list = []
for x in effectorp_busco_overlap:
    p_effectorp_list.remove(x)
for seq in p_effectorp_seq_list:
    if seq.id not in effectorp_busco_overlap:
        updated_effectorp_seq_list.append(seq)
print("This is the number of effector candidates after removal of BUSCOs: %i" % len(p_effectorp_list))

['evm.model.pcontig_041.167', 'evm.model.pcontig_042.2']
This is the number of effector candidates before removal: 1071
This is the number of effector candidates after removal of BUSCOs: 1069


In [15]:
#write out the sets of effector candidates
fh = open(p_effector_file, 'w')
for ec in set(p_effector_list):
    print(ec, file=fh)
fh.close()


#write out effector fasta files
SeqIO.write(updated_effector_seq_list, open(p_effector_file.replace('.list', '.protein.fasta'), 'w'), 'fasta')


1572

In [16]:
#write out the sets of effector candidates
fh = open(p_effectorp_file, 'w')
for ec in set(p_effectorp_list):
    print(ec, file=fh)
fh.close()


#write out effector fasta files
SeqIO.write(updated_effectorp_seq_list, open(p_effectorp_file.replace('.list', '.protein.fasta'), 'w'), 'fasta')


1069

In [17]:
#subset the gff files as well and write those out
p_gff_file = [os.path.join(GFF_FOLDER, x) for x in os.listdir(GFF_FOLDER)\
                 if x.startswith(genome+'p_ctg') and x.endswith('anno.gff3') ][0]

In [18]:
#get repeat gff files
p_repeat_gff_fn = [os.path.join(GFF_FOLDER, x) for x in os.listdir(GFF_FOLDER)\
                 if x.startswith(genome+'p_ctg') and x.endswith('REPET.gff') ][0]

In the command line in /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/TE_analysis the superfamily gff files were sorted as followed and copied over into the lists folder
sort Pst_104E_v12_h_ctg.REPET.superfamily.gff -k1,1n -k4,4n > Pst_104E_v12_h_ctg.REPET.sorted.superfamily.gff

In [19]:
#get repeat gff files
p_repeat_superfamily_gff_fn = [os.path.join(OUT_FOLDER, x) for x in os.listdir(OUT_FOLDER)\
                 if x.startswith(genome+'p_ctg') and x.endswith('REPET.sorted.superfamily.gff') ][0]


In [20]:
#gff header 
gff_header = ['seqid', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']

In [21]:
#now subset the gff files for effectors only
p_gff_df = pd.read_csv(p_gff_file, header = None, sep='\t', names= gff_header)
p_gff_df['ID'] = p_gff_df.attributes.str.extract(r'ID=([^;]*);', expand=False)
p_gff_df.sort_values(by=['seqid', 'start'], inplace = True)


In [22]:
#now sort REPET gff and write out again
p_repeat_gff_df = pd.read_csv(p_repeat_gff_fn, header=None, sep='\t', names=gff_header, comment='#')
p_repeat_gff_fn = os.path.join(OUT_FOLDER,p_repeat_gff_fn.split('/')[-1] )
p_repeat_gff_df.sort_values(by=['seqid', 'start']).to_csv(p_repeat_gff_fn, header=None, index=None, sep='\t')


In [23]:
#now write out dataframes for p_gff

#bed 6 file
p_effector_bed_fn = p_effector_file.replace('.list', '.gene.bed')
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_effector_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_effector_bed_fn, header=None, index=None, sep='\t')
    
p_effector_gff_fn = p_effector_file.replace('.list', '.gene.gff3')    
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_effector_list))].\
    loc[:,gff_header].to_csv(p_effector_gff_fn, header=None, index=None, sep='\t')


#bed 6 file no effectors
p_noeffector_bed_fn = p_effector_file.replace('p_effector.list', 'p_noeffector.gene.bed')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_effector_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_noeffector_bed_fn, header=None, index=None, sep='\t')

#no effector gff and list
p_noeffector_gff_fn = p_effector_file.replace('p_effector.list', 'p_noeffector.gene.gff3')    
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_effector_list))].\
    loc[:,gff_header].to_csv(p_noeffector_gff_fn, header=None, index=None, sep='\t')

file_name = p_effector_file.replace('p_effector.list', 'p_noeffector.list')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_effector_list))]['ID'].str.replace('TU', 'model')\
    .to_csv(file_name, header=None, index=None, sep='\t')

In [24]:
#now write out dataframes for p_gff

#bed 6 file
p_effectorp_bed_fn = p_effectorp_file.replace('.list', '.gene.bed')
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_effectorp_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_effectorp_bed_fn, header=None, index=None, sep='\t')
    
p_effectorp_gff_fn = p_effectorp_file.replace('.list', '.gene.gff3')    
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_effectorp_list))].\
    loc[:,gff_header].to_csv(p_effectorp_gff_fn, header=None, index=None, sep='\t')


#bed 6 file no effectors
p_noeffectorp_bed_fn = p_effectorp_file.replace('p_effectorp.list', 'p_noeffectorp.gene.bed')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_effectorp_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_noeffectorp_bed_fn, header=None, index=None, sep='\t')

#no effector gff and list
p_noeffectorp_gff_fn = p_effectorp_file.replace('p_effectorp.list', 'p_noeffectorp.gene.gff3')    
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_effectorp_list))].\
    loc[:,gff_header].to_csv(p_noeffectorp_gff_fn, header=None, index=None, sep='\t')

file_name = p_effectorp_file.replace('p_effectorp.list', 'p_noeffectorp.list')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_effectorp_list))]['ID'].str.replace('TU', 'model')\
    .to_csv(file_name, header=None, index=None, sep='\t')

In [25]:
#write out BUSCO for pcontigs
file_name = p_effector_file.replace('effector.list', 'busco.list')
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))]['ID'].str.replace('TU', 'model')\
    .to_csv(file_name, header=None, index=None, sep='\t')
    
p_busco_gff_fn = p_effector_file.replace('effector.list', 'busco.gene.gff3')
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))].\
    loc[:,gff_header].to_csv(p_busco_gff_fn, header=None, index=None, sep='\t')

p_busco_bed_fn = p_effector_file.replace('effector.list', 'busco.gene.bed')
p_gff_df[(p_gff_df.type == 'gene') & (p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_busco_bed_fn, header=None, index=None, sep='\t')

In [26]:
#write out non BUSCO for pcontigs
file_name = p_effector_file.replace('effector.list', 'non_busco.list')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))]['ID'].str.replace('TU', 'model')\
    .to_csv(file_name, header=None, index=None, sep='\t')
p_non_busco_gff_fn = p_effector_file.replace('effector.list', 'non_busco.gene.gff3')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))].\
    loc[:,gff_header].to_csv(p_non_busco_gff_fn, header=None, index=None, sep='\t')

p_non_busco_bed_fn = p_effector_file.replace('effector.list', 'non_busco.gene.bed')
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_non_busco_bed_fn, header=None, index=None, sep='\t')

In [27]:
#write out no busco no effector bed files
p_noeffector_nobusco_bed_fn = p_effector_file.replace('effector.list', 'no_busco_no_effector.gene.bed')    
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))\
         &(~p_gff_df.ID.str.replace('TU', 'model').isin(p_effector_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_noeffector_nobusco_bed_fn, header=None, index=None, sep='\t')


In [28]:
#write out no busco no effector bed files
p_noeffectorp_nobusco_bed_fn = p_effectorp_file.replace('effectorp.list', 'no_busco_no_effectorp.gene.bed')    
p_gff_df[(p_gff_df.type == 'gene') & (~p_gff_df.ID.str.replace('TU', 'model').isin(p_busco_list))\
         &(~p_gff_df.ID.str.replace('TU', 'model').isin(p_effectorp_list))].\
    loc[:,['seqid', 'start', 'end', 'ID', 'score', 'strand']].to_csv(p_noeffectorp_nobusco_bed_fn, header=None, index=None, sep='\t')


Ran up to here for now

In [ ]:
#get the distances

#read in files and generate bedtool objects. Issue might be that same variable are used for the bed files
#while those are sometimes gff and sometimes bed. Needs changing
p_effector_bed = BedTool(p_effector_gff_fn)
p_noeffector_bed = BedTool(p_noeffector_gff_fn)
p_busco_bed = BedTool(p_busco_gff_fn)
p_repeats_bed = BedTool(p_repeat_superfamily_gff_fn)
p_non_busco_bed = BedTool(p_non_busco_bed_fn)


#get closest repeat and make a df out of it
p_closest_rep_to_eff = p_effector_bed.closest(p_repeats_bed, d=True)
p_closest_rep_to_eff_df = p_closest_rep_to_eff.to_dataframe()

#here subset the repeat superfamily GFF to filder out smaller repeats if wanted
tmp_REPET = pd.read_csv(p_repeat_superfamily_gff_fn, header=None, sep='\t', names=gff_header)
tmp_REPET['distance'] = tmp_REPET.end - tmp_REPET.start
tmp_fn = p_repeat_superfamily_gff_fn.replace('superfamily', 'tmpsuperfamily')
tmp_REPET[tmp_REPET.distance > 400].loc[:,gff_header].to_csv(tmp_fn, header=None, sep='\t', index=None)

#read this in as repeat_df
p_repeats_bed = BedTool(tmp_fn)

In [ ]:
#bed closest header when using gff as input files
bed_repeat_closest_header = [x +'_gene' for x in gff_header] + [x +'_repeat' for x in gff_header] + ['distance']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print('Effectors closest Repeat')
print(p_effector_bed.closest(p_repeats_bed,d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_effector_bed.closest(p_repeats_bed, d=True, t='last').to_dataframe().boxplot(column=18)
plt.ylim(0, 10000)

In [ ]:
print('Repeat closest effector')
print(p_repeats_bed.closest(p_effector_bed,d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_repeats_bed.closest(p_effector_bed, d=True, t='last').to_dataframe().boxplot(column=18)


In [ ]:
print('BUSCO distance to closest repeat')
print(p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().boxplot(column=18)
plt.ylim(0, 10000)

In [ ]:
print('BUSCO distance to closest repeat')
print(p_repeats_bed.closest(p_busco_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_repeats_bed.closest(p_busco_bed, d=True,t='last').to_dataframe().boxplot(column=18)


In [ ]:
print('No Effector distance to closest repeat')
print(p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().boxplot(column=18)
plt.ylim(0, 10000)

In [ ]:
print('Relative distance from TE elements to other elements')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_repeats_bed.reldist(p_effector_bed)['reldist'], p_repeats_bed.reldist(p_effector_bed)['fraction'], label ='effectors')
plt.plot(p_repeats_bed.reldist(p_busco_bed)['reldist'], p_repeats_bed.reldist(p_busco_bed)['fraction'], label= 'busco')
plt.plot(p_repeats_bed.reldist(p_noeffector_bed)['reldist'], p_repeats_bed.reldist(p_noeffector_bed)['fraction'], label='no_effectors')
plt.ylim(0, 0.05)
plt.legend()

In [ ]:
print('Relative distance of other elements to TEs')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_effector_bed.reldist(p_repeats_bed)['reldist'], p_effector_bed.reldist(p_repeats_bed)['fraction'], label ='effectors')
plt.plot(p_busco_bed.reldist(p_repeats_bed)['reldist'], p_busco_bed.reldist(p_repeats_bed)['fraction'], label= 'busco')
plt.plot(p_noeffector_bed.reldist(p_repeats_bed)['reldist'], p_noeffector_bed.reldist(p_repeats_bed)['fraction'], label='no_effectors')
plt.ylim(0, 0.05)
plt.legend()

In [ ]:
#some stuff in the following lines

In [ ]:
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().groupby(17).count()[0].sort_values().tail(10)/len(p_noeffector_bed)*100

In [ ]:
p_effector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().groupby(17).count()[0].sort_values().tail(10)/len(p_effector_bed)*100

In [ ]:
p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().groupby(17).count()[0].sort_values().tail(10)/len(p_busco_bed)*100

In [ ]:
p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().groupby(17).mean()[18].sort_values().tail(10)

In [ ]:
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().groupby(17).mean()[18].sort_values().tail(10)

In [ ]:
p_effector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().groupby(17).mean()[18].sort_values().tail(10)

In [ ]:
#now lookine at ClassII:?:? specifically
tmp_REPET[(tmp_REPET.distance > 400)&(tmp_REPET.attributes == 'ClassII:?:?')].loc[:,gff_header].to_csv(tmp_fn, header=None, sep='\t', index=None)
p_repeats_bed = BedTool(tmp_fn)

In [ ]:
print('Effectors and there closest ClassII:?:? elements')
print(p_effector_bed.closest(p_repeats_bed,d=True,t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_effector_bed.closest(p_repeats_bed, d=True, t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print('ClassII:?:? elements and there closest Effectors ')
print(p_repeats_bed.closest(p_effector_bed, d=True, t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_repeats_bed.closest(p_effector_bed, d=True, t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print('Relative distance from ClassII:?:? elements and other elements')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_repeats_bed.reldist(p_effector_bed)['reldist'], p_repeats_bed.reldist(p_effector_bed)['fraction'], label ='effectors')
plt.plot(p_repeats_bed.reldist(p_busco_bed)['reldist'], p_repeats_bed.reldist(p_busco_bed)['fraction'], label= 'busco')
plt.plot(p_repeats_bed.reldist(p_noeffector_bed)['reldist'], p_repeats_bed.reldist(p_noeffector_bed)['fraction'], label='no_effectors')
plt.ylim(0, 0.05)
plt.legend()

In [ ]:
print('Relative distance of other elements to ClassII:?:?')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_effector_bed.reldist(p_repeats_bed)['reldist'], p_effector_bed.reldist(p_repeats_bed)['fraction'], label ='effectors')
plt.plot(p_busco_bed.reldist(p_repeats_bed)['reldist'], p_busco_bed.reldist(p_repeats_bed)['fraction'], label= 'busco')
plt.plot(p_noeffector_bed.reldist(p_repeats_bed)['reldist'], p_noeffector_bed.reldist(p_repeats_bed)['fraction'], label='no_effectors')
plt.ylim(0, 0.05)
plt.legend()

In [ ]:
print('BUSCOs distance to closest ClassII:?:?')
print(p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print('No effectors distance to closest ClassII:?:?')
print(p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().iloc[:,18:20].describe())
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().boxplot(column=18)

In [ ]:
print('No effectors distance to closest ClassII:?:? while allowing for overlaps')
print(p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().iloc[:,18:20].describe())
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().boxplot(column=18)

In [ ]:
#look at if effectors and buscos are linked or further apart using the redist function
print('Relative distance of between BUSCOS and effectors')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_effector_bed.reldist(p_busco_bed)['reldist'], p_effector_bed.reldist(p_busco_bed)['fraction'], label ='Effectors vs. BUSCO')
plt.plot(p_busco_bed.reldist(p_effector_bed)['reldist'], p_busco_bed.reldist(p_effector_bed)['fraction'], label= 'BUSCO vs effectors')

plt.ylim(0, 0.05)
plt.legend()

In [ ]:
print('Relative distance of between non-effectors and effectors')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_effector_bed.reldist(p_noeffector_bed)['reldist'], p_effector_bed.reldist(p_noeffector_bed)['fraction'], label ='Effectors vs. No effectors')
plt.plot(p_noeffector_bed.reldist(p_effector_bed)['reldist'], p_noeffector_bed.reldist(p_effector_bed)['fraction'], label= 'No effectors vs effectors')

plt.ylim(0, 0.05)
plt.legend()

In [ ]:
print('Relative distance of between non-effectors and effectors')
fig, ax = plt.subplots(figsize=(7,7))
plt.plot(p_busco_bed.reldist(p_non_busco_bed)['reldist'], p_busco_bed.reldist(p_non_busco_bed)['fraction'], label ='Effectors vs. No effectors')
plt.plot(p_non_busco_bed.reldist(p_busco_bed)['reldist'], p_non_busco_bed.reldist(p_busco_bed)['fraction'], label= 'No effectors vs effectors')

plt.ylim(0, 0.05)
plt.legend()

In [ ]:
#densities without overlaps between repeats and genes
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().loc[:,18].plot(kind='density')
p_effector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().loc[:,18].plot(kind='density', color='r')
p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe().loc[:,18].plot(kind='density', color='g')
plt.semilogx()

In [ ]:
#densities with overlaps between repeats and genes
p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().loc[:,18].plot(kind='density')
p_effector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().loc[:,18].plot(kind='density', color='r')
p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe().loc[:,18].plot(kind='density', color='g')
plt.semilogx()

In [ ]:
#define repeats
tmp_REPET = pd.read_csv(p_repeat_superfamily_gff_fn, header=None, sep='\t', names=gff_header)
tmp_REPET['distance'] = tmp_REPET.end - tmp_REPET.start
tmp_fn = p_repeat_superfamily_gff_fn.replace('superfamily', 'tmpsuperfamily')
tmp_REPET[tmp_REPET.distance > 400].loc[:,gff_header].to_csv(tmp_fn, header=None, sep='\t', index=None)

p_repeats_bed = BedTool(tmp_fn)

In [ ]:
#summary of closest TEs not allowing for overlaps 
#####Needs fixing for genes that don't have a nearest neighbou######
p_br_closest_df = p_busco_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe(names = bed_repeat_closest_header)

p_br_closest_pt = p_br_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_br_closest_pt['superfamily_%'] = p_br_closest_pt.count_nonzero / p_br_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_br_closest_pt.columns] 
new_columns = [x+'_busco' for x in p_br_closest_pt.columns]
p_br_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_er_closest_df = p_effector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe(names = bed_repeat_closest_header)

p_er_closest_pt = p_er_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_er_closest_pt['superfamily_%'] = p_er_closest_pt.count_nonzero / p_er_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_er_closest_pt.columns] 
new_columns = [x+'_effector' for x in p_er_closest_pt.columns]
p_er_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_ner_closest_df = p_noeffector_bed.closest(p_repeats_bed, d=True,t='last', io=True).to_dataframe(names = bed_repeat_closest_header)

p_ner_closest_pt = p_ner_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_ner_closest_pt['superfamily_%'] = p_ner_closest_pt['count_nonzero'] / p_ner_closest_pt['count_nonzero'].sum() *100
old_columns =[x for x in p_ner_closest_pt.columns] 
new_columns = [x+'_noeffector' for x in p_ner_closest_pt.columns]
p_ner_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

pd.options.display.float_format = '{:.2f}'.format

repeat_distance_df = pd.concat([p_ner_closest_pt.iloc[:,1:], p_br_closest_pt.iloc[:,1:], p_er_closest_pt.iloc[:,1:]], axis=1)

repeat_distance_df[repeat_distance_df['superfamily_%_effector'] >1].sort_values('superfamily_%_effector')

In [ ]:
#summary of closest TEs allowing for overlaps
p_br_closest_df = p_busco_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe(names = bed_repeat_closest_header)

p_br_closest_pt = p_br_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_br_closest_pt['superfamily_%'] = p_br_closest_pt.count_nonzero / p_br_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_br_closest_pt.columns] 
new_columns = [x+'_busco' for x in p_br_closest_pt.columns]
p_br_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_er_closest_df = p_effector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe(names = bed_repeat_closest_header)

p_er_closest_pt = p_er_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_er_closest_pt['superfamily_%'] = p_er_closest_pt.count_nonzero / p_er_closest_pt.count_nonzero.sum() *100
old_columns =[x for x in p_er_closest_pt.columns] 
new_columns = [x+'_effector' for x in p_er_closest_pt.columns]
p_er_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

p_ner_closest_df = p_noeffector_bed.closest(p_repeats_bed, d=True,t='last').to_dataframe(names = bed_repeat_closest_header)

p_ner_closest_pt = p_ner_closest_df.pivot_table(values='distance', index=['attributes_repeat'] ,aggfunc=[ np.count_nonzero, np.mean]).T.unstack().T

p_ner_closest_pt['superfamily_%'] = p_ner_closest_pt['count_nonzero'] / p_ner_closest_pt['count_nonzero'].sum() *100
old_columns =[x for x in p_ner_closest_pt.columns] 
new_columns = [x+'_noeffector' for x in p_ner_closest_pt.columns]
p_ner_closest_pt.rename(columns=dict(zip(old_columns,new_columns)), inplace = True)

pd.options.display.float_format = '{:.2f}'.format

repeat_distance_df = pd.concat([p_ner_closest_pt.iloc[:,1:], p_br_closest_pt.iloc[:,1:], p_er_closest_pt.iloc[:,1:]], axis=1)

repeat_distance_df[repeat_distance_df['superfamily_%_effector'] >1].sort_values('superfamily_%_effector')

Add some sub_sets for randommization to get equal sized groups

In [ ]:
#set the size of the subset here
sub_set = len(p_busco_bed)

In [ ]:
p_effector_bed = BedTool(p_effector_bed_fn)
p_allgene_bed = BedTool(p_gene_bed_fn)
p_busco_bed = BedTool(p_busco_bed_fn)
p_allall_rand_sub = p_allgene_bed.random_subset(sub_set)
p_effector_bed_rand_sub = p_effector_bed.random_subset(sub_set)
p_busco_bed_rand_sub = p_busco_bed.random_subset(sub_set)

In [ ]:
p_non_busco_non_effector_bed_fn

In [ ]:
#get some randomly subsampled beds of non_busco_non_effectors
p_non_busco_non_effector_bed = BedTool(p_non_busco_non_effector_bed_fn)
p_no_b_no_e_rand_sub = p_non_busco_non_effector_bed.random_subset(sub_set)
for n in range(0,10):
    fn_n = p_non_busco_non_effector_bed_fn.replace('.gene.bed', '.randsub_%s.gene.bed'%n)
    p_no_b_no_e_rand_sub = p_non_busco_non_effector_bed.random_subset(sub_set)
    p_no_b_no_e_rand_sub.to_dataframe().to_csv(fn_n, sep='\t', header=None, index=None)


In [ ]:
#get the distances with nearest gene effectors vs effectors
p_eself = p_effector_bed_rand_sub.closest(p_effector_bed_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_eself = p_eself[p_eself > -1]
p_eself.name = 'Effectors'
p_eall = p_effector_bed_rand_sub.closest(p_allall_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_eall= p_eall[p_eall > -1]
print(p_eself.describe())
p_eself.plot(kind='box')

In [ ]:
#get the distances with nearest gene all vs all subsampled
p_allall = p_allall_rand_sub.closest(p_allall_rand_sub, d=True, N=True).to_dataframe().iloc[:,12]
p_allall = p_allall[p_allall > -1]
p_allall.name = 'All_genes'
print(p_allall.describe())
p_allall.plot(kind='box')

In [ ]:
#now with buscos
p_bself = p_busco_bed_rand_sub.closest(p_busco_bed_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_bself = p_bself[p_bself > -1]
p_bself.name = 'BUSCO'
print(p_bself.describe())
p_bself.plot(kind='box')

In [ ]:
#non_effectors
p_noeffector_bed= BedTool(p_noeffector_bed_fn)
p_noeffector_rand_sub = p_noeffector_bed.random_subset(sub_set)
p_neself = p_noeffector_rand_sub.closest(p_noeffector_rand_sub, d=True,  N=True).to_dataframe().iloc[:,12]
p_neself = p_neself[p_neself > -1]
p_neself.name = 'No_effectors'
print(p_neself.describe())
p_neself.plot(kind='box')

Look if effectors and BUSCOs are actually linked
The question is if effectors are close to buscos than other genes. So do busco and effectors link together.

-> are busco more likely closest genes to effectors and vice versa
-> are the distance between busco and random genes larger than busco and effectors

In [ ]:
print('effectors distance to closest Busco')
p_effector_bed_rand_sub.closest(p_busco_bed_rand_sub, d=True,t='last', io=True).to_dataframe().boxplot(column=12)
effector_busco_c_df = p_effector_bed_rand_sub.closest(p_busco_bed_rand_sub, d=True,t='last', io=True).to_dataframe().iloc[:,12]
effector_busco_c_df = effector_busco_c_df[effector_busco_c_df>-1]
effector_busco_c_df.name = "Closest Busco to effector"
effector_busco_c_df.describe()

In [ ]:
print('Closest effector to busco')
p_busco_bed_rand_sub.closest(p_effector_bed_rand_sub, d=True,t='last', io=True).to_dataframe().boxplot(column=12)
busco_effector_c_df = p_busco_bed_rand_sub.closest(p_effector_bed_rand_sub, d=True,t='last', io=True).to_dataframe().iloc[:,12]
busco_effector_c_df = busco_effector_c_df[busco_effector_c_df>-1]
busco_effector_c_df.name = 'Closest effector to busco'
busco_effector_c_df.describe()

In [ ]:
print('effectors distance to closest rand')
p_effector_bed_rand_sub.closest(p_allall_rand_sub, d=True,t='last', io=True).to_dataframe().boxplot(column=12)
effector_randsubset_c_df = p_effector_bed_rand_sub.closest(p_allall_rand_sub, d=True,t='last', io=True).to_dataframe().iloc[:,12]
effector_randsubset_c_df = effector_randsubset_c_df[effector_randsubset_c_df>-1]
effector_randsubset_c_df.name =  "Closest randsubset to effector"
effector_randsubset_c_df.describe()

In [ ]:
print('busco distance to closest rand')
p_busco_bed_rand_sub.closest(p_allall_rand_sub, d=True,t='last', io=True).to_dataframe().boxplot(column=12)
busco_rand_sub_c_df = p_busco_bed_rand_sub.closest(p_allall_rand_sub, d=True,t='last', io=True).to_dataframe().iloc[:,12]
busco_rand_sub_c_df = busco_rand_sub_c_df[busco_rand_sub_c_df>-1]
busco_rand_sub_c_df.name = "Closest rand sub to BUSCO"
busco_rand_sub_c_df.describe()

In [ ]:
print('rand to closest busco')
p_allall_rand_sub.closest(p_busco_bed_rand_sub, d=True,t='last', io=True).to_dataframe().boxplot(column=12)
rand_sub_busco_c_df = p_allall_rand_sub.closest(p_busco_bed_rand_sub, d=True,t='last', io=True).to_dataframe().iloc[:,12]
rand_sub_busco_c_df = rand_sub_busco_c_df[rand_sub_busco_c_df>-1]
rand_sub_busco_c_df.name = 'Closest BUSCO to rand subset'
rand_sub_busco_c_df.describe()

In [ ]:
print('Rand to closest effector')
p_allall_rand_sub.closest(p_effector_bed_rand_sub, d=True,t='last', io=True).to_dataframe().boxplot(column=12)
rand_subset_effector_c_df = p_allall_rand_sub.closest(p_effector_bed_rand_sub, d=True,t='last', io=True).to_dataframe().iloc[:,12]
rand_subset_effector_c_df = rand_subset_effector_c_df[rand_subset_effector_c_df>-1]
rand_subset_effector_c_df.name ='Closest effector to rand subset'
rand_subset_effector_c_df.describe()

In [ ]:
#all against others both ways to see if effectors are closer to buscos compared to random subset
all_vs_others_c_df = \
 pd.concat([busco_effector_c_df,rand_subset_effector_c_df,effector_busco_c_df,rand_sub_busco_c_df,  \
             effector_randsubset_c_df, busco_rand_sub_c_df ], axis=1)

In [ ]:
fig, ax = plt.subplots(figsize=(15,6))
sns.violinplot(data=all_vs_others_c_df)
plt.xticks(rotation=-45)

In [ ]:
#define a function that subsets a dataframe to the inner quantil residual columnwise
def quant_cut_df(dataframe):
    nn_df = dataframe.copy()
    iqr_df_low = nn_df.apply(lambda x: x.quantile(0.25) - 1.5*(x.quantile(0.75) - x.quantile(0.25)) )
    iqr_df_low.name ='low'
    iqr_df_high = nn_df.apply(lambda x: x.quantile(0.75) + 1.5*(x.quantile(0.75) - x.quantile(0.25)) )
    iqr_df_high.name = 'high'

    iqr_df = pd.concat([iqr_df_low, iqr_df_high], axis=1).T

    iqr_nn_df = nn_df.apply(lambda x: x[(x > iqr_df.loc['low', x.name]) & (x  < iqr_df.loc['high', x.name])], axis=0)
    return iqr_nn_df 



In [ ]:
#plot the violine plots for distances once teh iqr is present only
all_vs_others_iqr_df = quant_cut_df(all_vs_others_c_df)
fig, ax = plt.subplots(figsize=(15,6))
sns.violinplot(data=all_vs_others_iqr_df)
plt.xticks(rotation=-45)

In [ ]:
all_vs_others_iqr_df.columns

In [ ]:
#do some stats on it look first into ranksum test
scipy.stats.ranksums(all_vs_others_c_df['Closest effector to busco'], all_vs_others_c_df['Closest effector to rand subset'])

In [ ]:
scipy.stats.ranksums(all_vs_others_c_df['Closest Busco to effector'], all_vs_others_c_df['Closest BUSCO to rand subset'])

In [ ]:
scipy.stats.mannwhitneyu(all_vs_others_c_df['Closest Busco to effector'], all_vs_others_c_df['Closest BUSCO to rand subset'],use_continuity=True, alternative='two-sided')

In [ ]:
scipy.stats.mannwhitneyu(all_vs_others_c_df['Closest effector to busco'], all_vs_others_c_df['Closest effector to rand subset'],use_continuity=True, alternative='two-sided')

In [ ]:
all_vs_others_iqr_melt = all_vs_others_iqr_df.melt()

In [ ]:
df = all_vs_others_iqr_melt\
    [all_vs_others_iqr_melt.variable.isin(all_vs_others_iqr_df.iloc[:,[0,1,2,3]].columns)].copy()
#do a boxplot and swarmplot on the same data
f, ax = plt.subplots(figsize=(15, 10))
#ax.set_xscale("log")
sns.violinplot(x="value", y="variable", data=df, cut=0,
          whis=np.inf)
plt.setp(ax.artists, alpha=.01)
sns.swarmplot(x="value", y="variable", data=df,
              size=2, color=".3", linewidth=0)
plt.xlim(0, 120000)
plt.ylabel("Comparisons")
plt.xlabel('Distance in bp')
ax.text(118000, 0.5, '*\n*\n*', color='k')
ax.plot([117000, 117000],[-0.1, 0.8], color ='k' )
ax.text(118000, 2.5, '*\n*\n*', color='k')
ax.plot([117000, 117000],[1.9, 2.8], color ='k' )
sns.despine(offset=10, trim=True)

In [ ]:
#do a boxplot on the same data using own bining
bins = [x for x in range(0, int(all_vs_others_iqr_df.max().max())+10000,2000 )]
all_vs_others_iqr_melt['bins'] = pd.cut(all_vs_others_iqr_melt.value, bins)
all_vs_others_iqr_hist = all_vs_others_iqr_melt.groupby(by=['variable', 'bins']).count().unstack().T

In [ ]:
all_vs_others_iqr_hist['bins'] = all_vs_others_iqr_hist.index.get_level_values(1)

all_vs_others_iqr_hist['bins_left'] = all_vs_others_iqr_hist['bins'].apply(lambda x: x.left)

In [ ]:
df = all_vs_others_iqr_hist.copy()
_max = all_vs_others_iqr_df.max().max()
fig, ax = plt.subplots(2,2, figsize=(10,15), sharex='all', sharey='all')
for axes, data in zip(ax.flat,  df.iloc[:,[1,0,2,3]].columns ):
    axes.bar(left=  df['bins_left'],height=df.loc[:,data],\
            color='k',width=1000)
    axes.set_title(data)
    
ax[1,0].set_xlabel('Distance to nearest neighbour')
ax[1,1].set_xlabel('Distance to nearest neighbour')
ax[0,0].set_ylabel('Frequency')
ax[1,0].set_ylabel('Frequency')
plt.xlim(-500, _max)

In [ ]:
#all_vs_others_iqr_hist.cumsum(axis=0)['Closest Busco to effector'] - all_vs_others_iqr_hist.cumsum(axis=0)['Closest BUSCO to rand subset']

In [ ]:
#all_vs_others_iqr_hist.cumsum(axis=0)['Closest effector to busco'] - all_vs_others_iqr_hist.cumsum(axis=0)['Closest effector to rand subset']

In [ ]:
#do a boxplot bin the data into same intervals for all and make new data frame out of it
all_vs_others_c_df_melt = all_vs_others_c_df.melt()
bins = [x for x in range(0, int(all_vs_others_c_df_melt.value.max())+10000,2000 )]
all_vs_others_c_df_melt['bins'] = pd.cut(all_vs_others_c_df_melt.value, bins)
all_vs_others_c_df_hist = all_vs_others_c_df_melt.groupby(by=['variable', 'bins']).count().unstack().T
all_vs_others_c_df_hist['bins'] = all_vs_others_c_df_hist.index.get_level_values(1)
all_vs_others_c_df_hist['bins_left'] = all_vs_others_c_df_hist['bins'].apply(lambda x: x.left)


In [ ]:
df = all_vs_others_c_df_hist.copy()
_max = all_vs_others_c_df_melt.value.max()
fig, ax = plt.subplots(2,2, figsize=(10,12), sharex='all', sharey='all')
for axes, data in zip(ax.flat,  df.iloc[:,[1,0,2,3]].columns ):
    axes.bar(left=  df['bins_left'],height=df.loc[:,data],\
            color='k',width=2000)
    axes.set_title(data)
    
ax[1,0].set_xlabel('Distance to nearest neighbour')
ax[1,1].set_xlabel('Distance to nearest neighbour')
ax[0,0].set_ylabel('Frequency')
ax[1,0].set_ylabel('Frequency')
plt.xlim(-500, _max)
#all_vs_others_iqr_df.hist(bins=30, figsize=(15,10), sharey='All', sharex="All")

cat Pst_104E_v12_ph_ctg.no_alleles_postQC.txt | xargs -I {} grep "{}" ../../enrichment_analysis/lists/Pst_104E_v12_p_effector.list > ../../enrichment_analysis/lists/Pst_104E_v12_p_effector.noalleles.list

In [ ]:
!ls {OUT_FOLDER}

In [ ]:
#check if effectors without alleles look any different.
orphan_effectors_list = pd.read_csv(os.path.join(OUT_FOLDER,'Pst_104E_v12_p_Cluster8_UnderDiversifyingSelection.list')\
                                   , sep='\t', header=None)[0].tolist()

In [ ]:
effector_list = pd.read_csv(os.path.join(OUT_FOLDER,'Pst_104E_v12_p_effector.list')\
                                   , sep='\t', header=None)[0].tolist()
busco_list = pd.read_csv(os.path.join(OUT_FOLDER,'Pst_104E_v12_p_busco.list')\
                                   , sep='\t', header=None)[0].tolist()

In [ ]:
count = 0
for x in effector_list:
    if x in busco_list:
        count = count +1
print(count)

In [ ]:
len(set(effector_list) -set(busco_list))

In [ ]:
test_df  = p_effector_bed_rand_sub.closest(p_busco_bed_rand_sub, d=True,t='last', io=True).to_dataframe()

In [ ]:
test_df[test_df[3].str.replace('TU', 'model').isin(orphan_effectors_list)].loc[:,12].describe()

In [ ]:
test_df.loc[:,12].describe()

No look a bit more into the distribution of the nearest neighbour distance

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(p_bself[p_bself < 10000], bins=20, rug = True, color='g',label='BUSCO')
sns.distplot(p_eself[p_eself < 10000], bins=20, rug = True, color='red',label = 'effector')
sns.distplot(p_neself[p_neself <10000], bins=20, rug=True, color='yellow', label ='not_effectors')
plt.legend()

In [ ]:
sns.distplot(p_bself[p_bself < 200000], bins=20, color='g' ,label='BUSCO')
sns.distplot(p_eself[p_eself < 200000], bins=20,  color='red',label = 'effector')
sns.distplot(p_neself[p_neself <200000], bins=20,  color='yellow', label ='not_effectors', axlabel='Distance to closest neighbour [bp]')
plt.legend()

In [ ]:
sns.distplot(p_bself, bins=20,color='g',  label='BUSCO')
sns.distplot(p_eself, bins=20,  color='red',label = 'effector')
sns.distplot(p_allall, bins=20,  color='b',label = 'all')
sns.distplot(p_neself, bins=20,  color='yellow', label ='not_effectors', axlabel='Distance to closest neighbour [bp]')
plt.legend()

In [ ]:
#now make a nearest neightbour dataframe
nn_df = pd.concat([p_allall, p_bself, p_eself], names=['All_genes', 'BUSCO', 'effectors'], axis=1)

In [ ]:
sns.violinplot(data=nn_df, palette="Set3")

In [ ]:
#NOW FILTER BY QUANTIELS

In [ ]:
low = 0
high = 0.8
quant_df = nn_df.quantile([low,high])

qfilt_nn_df = nn_df.apply(lambda x: x[(x > quant_df.loc[low, x.name]) & (x  < quant_df.loc[high, x.name])], axis=0)

sns.violinplot(data=qfilt_nn_df , palette="Set3")

In [ ]:
#now filter on IQR

In [ ]:
iqr_df_low = nn_df.apply(lambda x: x.quantile(0.25) - 1.5*(x.quantile(0.75) - x.quantile(0.25)) )
iqr_df_low.name ='low'
iqr_df_high = nn_df.apply(lambda x: x.quantile(0.75) + 1.5*(x.quantile(0.75) - x.quantile(0.25)) )
iqr_df_high.name = 'high'

iqr_df = pd.concat([iqr_df_low, iqr_df_high], axis=1).T

iqr_nn_df = nn_df.apply(lambda x: x[(x > iqr_df.loc['low', x.name]) & (x  < iqr_df.loc['high', x.name])], axis=0)
plt.title('Violine plot of nearest neighbour in the same category')
sns.violinplot(data=iqr_nn_df  , palette="Set3")

In [ ]:
p_bself[p_bself < 20000].plot(kind='density')
p_eself[p_eself < 20000].plot(kind='density', color='g')
p_neself[p_neself < 20000].plot(kind='density', color='r')
p_eall[p_eall < 20000].plot(kind='density', color='y')
#plt.semilogx()
print(len(p_bself[p_bself < 20000]), len(p_eself[p_eself < 20000]), len(p_neself[p_neself < 20000]), len(p_eall[p_eall < 20000]))
plt.xlim(0, 20000)

In [ ]:
p_eall[p_eall].plot(kind='hist', bins=20, normed=True)
plt.ylim(0, 0.00020)

In [ ]:
p_eself[p_eself].plot(kind='hist', bins=20, normed=True)
plt.ylim(0, 0.00020)

In [ ]:
p_bself[p_bself ].plot(kind='hist', bins=20,normed=True)
plt.ylim(0, 0.00020)

In [ ]:
p_neself[p_neself ].plot(kind='hist',bins=20,normed=True)
plt.ylim(0, 0.00020)

It seems like there seems to be no clear link between effector candidates and closest neighbour in terms of TEs in general. They all have the same distance in general.
Maybe Gypsy and ClassII:?:? should be looked at more carefully. Those are depleted and enriched in busco and effector genes. In general there seems to be a trend towards ClassII elements compared to ClassI in effector candidates.
This changes when for allowing for overlaps.

It seems that effectors are closer together than noneffector genes. Buscos also seem to cluster a bit. Let's see if we can visualize the location of genes on certain contigs vs repeats and such.

In [ ]:
#start with getting effectors per contig divided by length divded by # of overall genes

In [ ]:
#get effectors per contig
effector_bdf = pd.read_csv(p_effector_bed_fn, header=None, sep='\t')

In [ ]:
p_effectors_per_contig  = effector_bdf.groupby(0).count()[1]
p_effectors_per_contig.name = 'effectors'

In [ ]:
#get all others counts per contig
p_all_genes_per_contig = pd.read_csv(p_gene_bed_fn, header=None, sep='\t').groupby(0).count()[1]
p_all_genes_per_contig.name = 'all_genes'
p_noeffectors_per_contig = pd.read_csv(p_noeffector_bed_fn, header=None, sep='\t').groupby(0).count()[1]
p_noeffectors_per_contig.name = 'no_effectors'
p_busco_per_contig = pd.read_csv(p_busco_bed_fn, header=None, sep='\t').groupby(0).count()[1]
p_busco_per_contig.name = 'buscos'

In [ ]:
#read in the genome file that contains the length of each contig in the second column
p_contig_length = pd.read_csv(os.path.join(GFF_FOLDER, 'Pst_104E_v12_p_ctg.genome_file'), header = None,\
                          names=['contig' , 'length'], sep='\t').sort_values('contig')

In [ ]:
p_contig_length.index = p_contig_length.contig

p_contig_length = p_contig_length.loc[:, 'length']

gene_dis_per_contig = pd.concat([p_contig_length,p_all_genes_per_contig,p_noeffectors_per_contig,p_busco_per_contig,  p_effectors_per_contig ], axis = 1)

In [ ]:
from scipy.stats import chisquare

In [ ]:
#fill nan with 0s
gene_dis_per_contig.fillna(value=0, inplace=True)

In [ ]:
gene_dis_per_contig.effectors.sum()

In [ ]:
#chisquare for genes per contig vs bases per contig
chisquare(gene_dis_per_contig.length.values/gene_dis_per_contig.length.sum()\
          , gene_dis_per_contig.all_genes.values/gene_dis_per_contig.all_genes.sum())

In [ ]:
chisquare(gene_dis_per_contig.buscos.values/gene_dis_per_contig.buscos.sum()\
          , gene_dis_per_contig.all_genes.values/gene_dis_per_contig.all_genes.sum())

In [ ]:
#calculate the % of each category that are on a specific contig
gene_dis_per_contig['%busco'] = gene_dis_per_contig.buscos.values/gene_dis_per_contig.buscos.sum() *100
gene_dis_per_contig['%effector'] = gene_dis_per_contig.effectors.values/gene_dis_per_contig.effectors.sum() *100
gene_dis_per_contig['%all_genes'] = gene_dis_per_contig.all_genes.values/gene_dis_per_contig.all_genes.sum() *100
gene_dis_per_contig['%no_effector'] = gene_dis_per_contig.no_effectors.values/gene_dis_per_contig.no_effectors.sum() *100

In [ ]:
#not sure why this isn't working
chisquare(gene_dis_per_contig['%effector'], f_exp = gene_dis_per_contig['%all_genes'])

In [ ]:
#have a look at the different percentages
gene_dis_per_contig.loc[:,['%effector', '%all_genes','%no_effector' ,'%busco']]

In [ ]:
#contigs which % effectors is more than % of all genes or of buscos
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) &\
                    (gene_dis_per_contig['%effector'] > gene_dis_per_contig['%busco'])].loc[:,['effectors', '%effector', '%all_genes']].index

In [ ]:
#contigs which % effectors is more than % of all genes or of buscos
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) &\
                    (gene_dis_per_contig['%effector'] > gene_dis_per_contig['%busco'])].loc[:,['effectors', '%effector', '%all_genes', '%busco']]

In [ ]:
%%R
setwd("/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/enrichment_analysis/lists")
getwd()
busco_bed <- import("Pst_104E_v12_p_busco.gene.bed", format ='bed')
effector_bed <- import("Pst_104E_v12_p_effector.gene.bed", format='bed')
no_effector_bed <- import("Pst_104E_v12_p_effectorp_noeffector.gene.bed", format='bed')
#Pst_104E_seqinfo <- import("../../../032017_assembly/Pst_104E_v12_p_ctg.genome_file", as = "Rle")
#Pst_104E_seqinfo
#effector_bed
pn.area <- 100
pn.dist <- 100
pn.jacc <- 100
effector_to_busco_bed <- GenometriCorr:::GenometriCorrelation(effector_bed, busco_bed,  chromosomes.to.proceed = c("pcontig_001"), ecdf.area.permut.number = pn.area,mean.distance.permut.number = pn.dist,jaccard.measure.permut.number = pn.jacc,keep.distributions = TRUE, showProgressBar = FALSE)
print(effector_to_busco_bed)

In [ ]:
%%R
GenometriCorr:::graphical.report(effector_to_busco_bed, pdffile = "../figures/Pst_104E_p_effector_to_busco_bed_pcontig_001.pdf",  show.all = TRUE)

In [ ]:
#this moves the file to a folder were jupyter can access it for displaying.
copyanything(OUT_FOLDER_FIG, TMP_FIG_PATH)

In [ ]:
PDF('Pst_104E_p_effector_to_no_effector.pdf')

In [ ]:
PDF('Pst_104E_p_effector_to_busco.pdf')

In [ ]:
PDF('Pst_104E_p_effector_to_busco_vis.pdf')

In [ ]:
PDF('Pst_104E_p_effector_to_no_effector_vis.pdf')

In [ ]:
#effectors per contig > than expected 1.5 increase?
gene_dis_per_contig[(gene_dis_per_contig['%effector'] / gene_dis_per_contig['%all_genes']) > 1.5].loc[:,['effectors', '%effector', 'all_genes','%all_genes']].index

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) &\
                    (gene_dis_per_contig['%effector'] > gene_dis_per_contig['%busco'])].sum()

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) &\
                    (gene_dis_per_contig['%all_genes'] > gene_dis_per_contig['%busco'])].sum()

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) ].sum()

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%busco'] > gene_dis_per_contig['%all_genes'])].sum()

In [ ]:
#sum off all effectors on enriched? contigs
gene_dis_per_contig[(gene_dis_per_contig['%effector'] > gene_dis_per_contig['%all_genes']) ].sum()

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.2) & (gene_dis_per_contig['%busco'] > 0.5 )\
                    ].loc[:,['buscos', '%busco', '%all_genes']]

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.2) \
                    ].sum()

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.1) & \
                   (gene_dis_per_contig['%effector'] / gene_dis_per_contig['%all_genes'] < 0.9) ].sum()

In [ ]:
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%all_genes'] > 1.0000001) & \
                   (gene_dis_per_contig['%effector'] / gene_dis_per_contig['%all_genes'] < 0.9999) ].sum()

In [ ]:
#enriched for effectors relative to busco and all genes. Seems to be the case that their might be contigs
#with less BUSCOs and more effectors
gene_dis_per_contig[(gene_dis_per_contig['%busco'] / gene_dis_per_contig['%effector'] < 1) & \
                    (gene_dis_per_contig['%all_genes'] / gene_dis_per_contig['%effector'] < 1) ].sum()

Get closest features genes vs. TEs

In [ ]:
tmp_REPET = pd.read_csv(p_repeat_superfamily_gff_fn, header=None, sep='\t', names=gff_header)
tmp_REPET['distance'] = tmp_REPET.end - tmp_REPET.start

tmp_fn = p_repeat_superfamily_gff_fn.replace('superfamily.gff', 'tmpsuperfamily.bed')
tmp_REPET[tmp_REPET.distance > 400].loc[:,['seqid', 'start', 'end', 'attributes', 'score', 'strand']].to_csv(tmp_fn, header=None, sep='\t', index=None)

p_repeats_bed = BedTool(tmp_fn)

In [ ]:
tmp_df = p_effector_bed.closest( [p_repeats_bed.fn,p_allgene_bed.fn] , mdb='all', d=True, N=True).to_dataframe()

tmp_df.rename(columns={10: 'ID'}, inplace=True)

print('Per of effectors having genes as closest feature %f2' % (tmp_df[tmp_df.ID.str.contains('evm')][0].count()/len(p_effector_bed)*100))

print('Per of effectors having TE as closest %f2' % (tmp_df[~tmp_df.ID.str.contains('evm')][0].count()/len(p_effector_bed)*100))


In [ ]:
tmp_df = p_allgene_bed.closest( [p_repeats_bed.fn,p_allgene_bed.fn] , mdb='all', d=True, N=True).to_dataframe()

tmp_df.rename(columns={10: 'ID'}, inplace=True)

print('Per of genes having genes as closest feature %f2' % (tmp_df[tmp_df.ID.str.contains('evm')][0].count()/len(p_allgene_bed)*100))

print('Per of genes having TE as closest %f2' % (tmp_df[~tmp_df.ID.str.contains('evm')][0].count()/len(p_allgene_bed)*100))

Get 5 and 3 prime distances of genes

In [ ]:
#now get the 3 prime distante to genes using the D='a' and iu flag in bedtools
g_to_g_3 = p_allgene_bed.closest( p_allgene_bed.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]


g_to_g_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)

#now get the 5 prime distante to genes using the D='a' and id flag in bedtools
g_to_g_5 = p_allgene_bed.closest( p_allgene_bed.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
g_to_g_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

g_to_g_merged = g_to_g_3.merge(g_to_g_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account. Filter out everythign that has a 5' distance of -1 meaning nothing
#and is at the begining of the contig .1 and at the 3' end drop everything that has a negative distance meaning no 3' neighbour

g_to_g_merged = g_to_g_merged[ (g_to_g_merged['5_target'] !='.') &(g_to_g_merged['3_target'] !='.') ]


g_to_g_merged['5_distance'] = abs(g_to_g_merged['5_distance'])
g_to_g_merged['5_distance_log10'] = np.log10(g_to_g_merged['5_distance'])
g_to_g_merged['3_distance_log10'] = np.log10(g_to_g_merged['3_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged, kind="kde")



#do the same for effectors
#now for effectors 
#getting 5' and 3' distance
e_to_g_3 = p_effector_bed.closest( p_allgene_bed.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
e_to_g_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
e_to_g_5 = p_effector_bed.closest( p_allgene_bed.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
e_to_g_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

#merging them
e_to_g_merged = e_to_g_3.merge(g_to_g_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges
e_to_g_merged = e_to_g_merged[((e_to_g_merged['5_target'] != '.') & ((e_to_g_merged['3_target'] != '.'))) ]
e_to_g_merged['5_distance'] = abs(e_to_g_merged['5_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged, kind="kde")


#now for busco
b_to_g_3 = p_busco_bed.closest( p_allgene_bed.fn ,  N=True, iu=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
b_to_g_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
b_to_g_5 = p_busco_bed.closest( p_allgene_bed.fn ,  N=True, id=True, D='a' ).to_dataframe().iloc[:,[0, 3, 9,12]]
b_to_g_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)
b_to_g_merged = b_to_g_3.merge(b_to_g_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges see above for details
b_to_g_merged = b_to_g_merged[((b_to_g_merged['5_target'] !='.') & (b_to_g_merged['3_target'] !='.')) ]
b_to_g_merged['5_distance'] = abs(b_to_g_merged['5_distance'])
sns.jointplot(x='3_distance', y='5_distance', data=b_to_g_merged, kind="kde")

#now start plotting stuff
#sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged, kind="kde", color='r', xlim=10000, ylim=10000)
#sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged, kind="kde", xlim=10000, ylim=10000)


#subset everything by fixed numbers maybe to IQR or such in future
sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged[(e_to_g_merged['5_distance'] < 10000) &\
        (e_to_g_merged['3_distance'] < 10000)], kind="hex", color='r',xlim=[0,10000], ylim=[0,10000],\
             marginal_kws=dict(bins=30))


sns.jointplot(x='3_distance', y='5_distance', data=b_to_g_merged[(b_to_g_merged['5_distance'] < 10000) &\
        (b_to_g_merged['3_distance'] < 10000)], kind="hex",color='g', xlim=[0,10000], ylim=[0,10000],\
              marginal_kws=dict(bins=30))

sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged[(g_to_g_merged['5_distance'] < 10000) &\
        (g_to_g_merged['3_distance'] < 10000)], kind="hex", xlim=[0,10000], ylim=[0,10000],\
              marginal_kws=dict(bins=30))

Now look at the distance to the closest gene of the same group

In [ ]:
#now get the 3 prime distante to genes using the D='a' and iu flag in bedtools
p_allall_rand_sub = p_allgene_bed.random_subset(sub_set)
p_effector_bed_rand_sub = p_effector_bed.random_subset(sub_set)
p_busco_bed_rand_sub = p_busco_bed.random_subset(sub_set)

all_all_rand_3 = p_allall_rand_sub.closest( p_allall_rand_sub ,  N=True, iu=True, D='a' ).to_dataframe()


all_all_rand_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)

#now get the 5 prime distante to genes using the D='a' and id flag in bedtools
all_all_rand_5 = p_allall_rand_sub.closest( p_allall_rand_sub.fn ,  N=True, id=True, D='a' ).to_dataframe()
all_all_rand_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

all_all_rand_merged = all_all_rand_3.merge(all_all_rand_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account. Filter out everythign that has a 5' distance of -1 meaning nothing
#and is at the begining of the contig .1 <- the .1 was no appropriate here as the random subsampling can lead
#to alternative first gene models selected on the respective contig
#extended the initial df of closest to include also queyr and target. Now filtering on no target ='.' possible

all_all_rand_merged = all_all_rand_merged[((all_all_rand_merged['5_target'] != '.')&(all_all_rand_merged['3_target'] != '.')  ) ]



all_all_rand_merged['5_distance'] = abs(all_all_rand_merged['5_distance'])
all_all_rand_merged['5_distance_log10'] = np.log10(all_all_rand_merged['5_distance'])
all_all_rand_merged['3_distance_log10'] = np.log10(all_all_rand_merged['3_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=all_all_rand_merged, kind="kde")



In [ ]:

#do the same for effectors
#now for effectors 
#getting 5' and 3' distance
e_to_e_sub_3 = p_effector_bed_rand_sub.closest( p_effector_bed_rand_sub ,  N=True, iu=True, D='a' ).to_dataframe()
e_to_e_sub_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
e_to_e_sub_5 = p_effector_bed_rand_sub.closest( p_effector_bed_rand_sub.fn ,  N=True, id=True, D='a' ).to_dataframe()
e_to_e_sub_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)

#merging them
e_to_e_sub_merged = e_to_e_sub_3.merge(e_to_e_sub_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges
e_to_e_sub_merged = e_to_e_sub_merged [((e_to_e_sub_merged ['5_target'] != '.') & (e_to_e_sub_merged ['3_target'] != '.') )  ]
e_to_e_sub_merged ['5_distance'] = abs(e_to_e_sub_merged ['5_distance'])

sns.jointplot(x='3_distance', y='5_distance', data=e_to_e_sub_merged , kind="kde", color='red')



In [ ]:
p_busco_bed_rand_sub.closest( p_busco_bed_rand_sub.fn ,  N=True, iu=True, D='a' ).to_dataframe().head()

In [ ]:

#now for busco
p_busco_bed_rand_sub_3 = p_busco_bed_rand_sub.closest( p_busco_bed_rand_sub.fn ,  N=True, iu=True, D='a' ).to_dataframe()
p_busco_bed_rand_sub_3.rename(columns={12:'3_distance', 3:'query', 9:'3_target', 0:'contig'}, inplace=True)
p_busco_bed_rand_sub_5 = p_busco_bed_rand_sub.closest( p_busco_bed_rand_sub.fn ,  N=True, id=True, D='a' ).to_dataframe()
p_busco_bed_rand_sub_5.rename(columns={12:'5_distance', 3:'query', 9:'5_target', 0:'contig'}, inplace=True)
p_busco_bed_rand_sub_merged = p_busco_bed_rand_sub_3.merge(p_busco_bed_rand_sub_5, on=['query', 'contig'])

#needs to be fixed to take boundaries into account
#remove genes on the edges see above for details
p_busco_bed_rand_sub_merged = p_busco_bed_rand_sub_merged[((p_busco_bed_rand_sub_merged['5_target'] != '.') & (p_busco_bed_rand_sub_merged['5_target'] != '.') )]
p_busco_bed_rand_sub_merged['5_distance'] = abs(p_busco_bed_rand_sub_merged['5_distance'])
sns.jointplot(x='3_distance', y='5_distance', data=p_busco_bed_rand_sub_merged, kind="kde" ,color = 'green')

#now start plotting stuff
#sns.jointplot(x='3_distance', y='5_distance', data=e_to_g_merged, kind="kde", color='r', xlim=10000, ylim=10000)
#sns.jointplot(x='3_distance', y='5_distance', data=g_to_g_merged, kind="kde", xlim=10000, ylim=10000)




In [ ]:
#some more plotting for when subsetting the dataframe to exclude outliers
sns.jointplot(x='3_distance', y='5_distance', data=p_busco_bed_rand_sub_merged[(p_busco_bed_rand_sub_merged['5_distance'] < 20000)&\
                                                                              (p_busco_bed_rand_sub_merged['3_distance'] < 20000)],\
              kind="kde" ,color = 'green')

In [ ]:
#some more plotting for when subsetting the dataframe to exclude outliers
sns.jointplot(x='3_distance', y='5_distance', data=e_to_e_sub_merged[(e_to_e_sub_merged['5_distance'] < 20000)&\
                                                                              (e_to_e_sub_merged['3_distance'] < 20000)],\
              kind="kde" ,color = 'red')

In [ ]:
#looking more into the distance distribution between effectors use the index for this purpose and groupby or filteirng by 
#contig
#getting 5' and 3' distance


In [ ]:
#getting 5' and 3' distance
e_to_e_sub_distance = e_to_e_sub_3.merge(e_to_e_sub_5, on = ['query','contig'])

#convert negative -1 from bedtools closest to nan and make values absolute
tmp_index = e_to_e_sub_distance[e_to_e_sub_distance['5_target'] == '.'].index.tolist()
e_to_e_sub_distance.loc[tmp_index, '5_distance'] = np.nan
e_to_e_sub_distance['5_distance'] = abs(e_to_e_sub_distance['5_distance'])
e_to_e_sub_3_distance['5_distance'].fillna(-1) 
#convert -1 from bedtools closest to nan in 3_distance
tmp_index = e_to_e_sub_distance[e_to_e_sub_distance['3_target'] == '.'].index.tolist()
e_to_e_sub_distance.loc[tmp_index, '3_distance'] = -1



#now subset the dataframe for a fixed distance 
max_distance = 15000
#subset the df and get the index first by distance and than by linkage (remember everything at the edges got a  -1 as distance)
e_to_e_less_d_df = e_to_e_sub_distance[\
                        ((e_to_e_sub_distance['3_distance'] <max_distance)&(e_to_e_sub_distance['3_distance'] > - 1)) \
                                       | 
                        ((e_to_e_sub_distance['5_distance'] <max_distance) &(e_to_e_sub_distance['5_distance'] > -1))  ]

#everything at the edges got a  -1 as distance


#get the max distance of two consective genes in teh e_to_e_less_d_df. It could be to have A-B close and C-D close but not B-C currently
#this would not have gotten caught.

e_to_e_less_d_df['next_distance'] = abs(e_to_e_less_d_df.shift(-1)['1_x'] - e_to_e_less_d_df['2_x'])

#now set everything that is not on consecutive contigs max_distance +1
nex_contig_index = e_to_e_less_d_df[e_to_e_less_d_df.shift(-1)['contig'] != e_to_e_less_d_df['contig']].index.values

e_to_e_less_d_df.loc[nex_contig_index, 'next_distance'] = max_distance +1

#get the index values and not the series
e_to_e_less_d_df_index = e_to_e_less_d_df.index.values

#transfer the next_distance of the linked once the main datframe and make everything else max_distance +1

e_to_e_sub_distance['next_linked_distance'] = max_distance +1

e_to_e_sub_distance.loc[e_to_e_less_d_df_index, 'next_linked_distance'] = e_to_e_less_d_df.next_distance


#introduce new column 'linked' and make this 1 were the genes are linked (e.g. less than max distance apart)
e_to_e_sub_distance['linked'] =0
e_to_e_sub_distance.loc[e_to_e_less_d_df_index, 'linked']  = 1
#get a new columns linkage_group that is 0 for now
e_to_e_sub_distance['linkage_group'] = 0

#get linkage groups first filtered by consecutive index
tmp_linkage_groups = ((e_to_e_sub_distance[e_to_e_sub_distance.linked == 1].linked \
                      != e_to_e_sub_distance[e_to_e_sub_distance.linked == 1].linked.index.to_series().diff().eq(1))).cumsum()\
#this also adds together genes that are not really linked because they are on a different contig or A-B close and C-D close but not B-C
e_to_e_sub_distance.loc[e_to_e_sub_distance[e_to_e_sub_distance.linked == 1].index, 'tmp_lg']  = tmp_linkage_groups

test = e_to_e_sub_distance[e_to_e_sub_distance.linked == 1]
#the indexes are consectutive as indicated by the tmp linkage_group
unlinked_lg_index = test[(test.tmp_lg.shift(-1) == test.tmp_lg)][(test[(test.tmp_lg.shift(-1) == test.tmp_lg)]\
                                              ['next_linked_distance'] > max_distance)]
unlinked_lg_index = (test[(test.tmp_lg.shift(-1) == test.tmp_lg)]\
                                              ['next_linked_distance'] > max_distance)
#combine this remove unlinked_lg_indexs from initial linkage group by making everything Flase that is not linked.
tmp_linkage_groups = ((e_to_e_sub_distance[e_to_e_sub_distance.linked == 1].linked \
                      != e_to_e_sub_distance[e_to_e_sub_distance.linked == 1].linked.index.to_series().diff().eq(1))\
| unlinked_lg_index.shift(1)).cumsum()
e_to_e_sub_distance.loc[e_to_e_sub_distance[e_to_e_sub_distance.linked == 1].index, 'linkage_group']  = tmp_linkage_groups
#now loop over the contigs and connect the consecutive linked effectors (e.g. linked == 1) with a cumsum 
#for contig in e_to_e_sub_distance['contig'].unique():
   # tmp_df = e_to_e_sub_distance[e_to_e_sub_distance.contig == contig]
    #check were the index difference is not one and add it up
    #tmp_linkage_groups = (tmp_df[tmp_df.linked == 1].linked != tmp_df[tmp_df.linked == 1].linked.index.to_series().diff().eq(1)).cumsum() 
    # e_to_e_sub_distance.loc[tmp_df[tmp_df.linked == 1].index, 'linkage_group'] =  tmp_linkage_groups
    #print(contig)
#e_to_e_sub_distance = e_to_e_sub_distance.loc[:,['contig', 'query', '3_target',\
                                        # '3_distance', '5_target', '5_distance', 'linked', 'linkage_group', 'next_linked_distance']]


e_to_e_sub_distance = e_to_e_sub_distance.loc[:,[ 'query', '3_target',\
                                         '5_target',  'linked', 'linkage_group', 'next_linked_distance']]
e_to_e_sub_distance['lg_freq'] = e_to_e_sub_distance.groupby('linkage_group')['linkage_group'].transform('count')

In [ ]:
#makeing a function out of 5' and 3' distance bining
def _5_to_3_chains(_5_prime_df, _3_prime_df, max_distance=15000, label=''):
    """This function takes two dataframes one from _5_prime_ nearest neighbour and one from
    _3_prime_nearest neighbour. max_distance and label for the df columns can be added as well.
    Those should be generated with pybedtools and converted to dataframe
    without subsetting and with selecting the distance.
    Columns should be renamed as
    12:'3_distance', 3:'query', 9:'3_target', 0:'contig' for _3_prime
    and
    12:'5_distance', 3:'query', 9:'5_target', 0:'contig' for _5_prime
    The output will be new dataframe that contains the merged dataframes, the linked information, the linkage group,
    and the frequency of each linkage group = group member count.
    And a dataframe that summarizes the bin size, number within each bin and number of each bin.
    member_count_label	bin_size_label	number_of_bins_label
    """
    five_df = _5_prime_df.copy()
    three_df = _3_prime_df.copy()
    max_distance = max_distance
    
    #getting 5' and 3' distance
    distance_df = three_df.merge(five_df, on = ['query','contig'])

    #convert negative -1 from bedtools closest to nan and make values absolute
    tmp_index = distance_df[distance_df['5_target'] == '.'].index
    distance_df.loc[tmp_index, '5_distance'] = np.nan
    distance_df['5_distance'] = abs(distance_df['5_distance'])
    distance_df['5_distance'].fillna(-1, inplace = True) 
    #convert -1 from bedtools closest to nan in 3_distance
    tmp_index = distance_df[distance_df['3_target'] == '.'].index
    distance_df.loc[tmp_index, '3_distance'] = -1

    #subset the df and get the index first by distance and than by linkage (remember everything at the edges got a  -1 as distance)
    sub_distance_df = distance_df[\
                            ((distance_df['3_distance'] <max_distance)&(distance_df['3_distance'] > - 1)) \
                                           | 
                            ((distance_df['5_distance'] <max_distance) &(distance_df['5_distance'] > -1))  ]

    #get the max distance of two consective genes in teh distance_df. It could be to have A-B close and C-D close but not B-C currently
    #this would not have gotten caught. #to be illustrated

    sub_distance_df['next_distance'] = abs(sub_distance_df.shift(-1)['1_x'] - sub_distance_df['2_x'])

    #now set the distance to the next gene to max_distance +1 if the next contig is different
    #from the current one
    next_contig_index = sub_distance_df[sub_distance_df.shift(-1)['contig'] != sub_distance_df['contig']].index.values

    sub_distance_df.loc[next_contig_index, 'next_distance'] = max_distance +1

    #get the index values and not the series
    sub_distance_df_index = sub_distance_df.index.values

    #transfer the next_distance of the linked once the main datframe and make everything else max_distance +1

    distance_df['next_linked_distance'] = max_distance +1

    distance_df.loc[sub_distance_df_index, 'next_linked_distance'] = sub_distance_df.next_distance


    #introduce new column 'linked' and make this 1 were the genes are linked (e.g. less than max distance apart)
    distance_df['linked'] =0
    distance_df.loc[sub_distance_df_index, 'linked']  = 1
    #get a new columns linkage_group that is 0 for now
    distance_df['linkage_group'] = 0

    #get linkage groups first filtered by consecutive index
    tmp_linkage_groups = ((distance_df[distance_df.linked == 1].linked \
                          != distance_df[distance_df.linked == 1].linked.index.to_series().diff().eq(1))).cumsum()\
    
    #this also adds together genes that are not really linked because they are on a different contig or A-B close and
    #C-D close but not B-C. We need to take care of this later on using the next_linked_distance column
    distance_df.loc[distance_df[distance_df.linked == 1].index, 'tmp_lg']  = tmp_linkage_groups
    
    #generate a new sub_distance_df that has all colmuns as the main distance df
    sub_distance_df = distance_df[distance_df.linked == 1]
    
    #the indexes are consectutive as indicated by the tmp linkage_group. 
    #now identify where linked sequences are separated by more than the max_distance. This includes intercontigs breaks.
    
    unlinked_lg_index = (sub_distance_df[(sub_distance_df.tmp_lg.shift(-1) == sub_distance_df.tmp_lg)]\
                                                  ['next_linked_distance'] > max_distance)
    
    #combine this remove unlinked_lg_indexs from initial linkage group by making everything Flase that is not linked.
    #this requires to 'add' the unlinked_lg_index boolean array to the consecutive boolean array using an or |
    #meaning only the Trues are transfered and this needs to be shiffted one downward (could have also done previous distance and not
    #next)
    
    tmp_linkage_groups = ((distance_df[distance_df.linked == 1].linked \
                          != distance_df[distance_df.linked == 1].linked.index.to_series().diff().eq(1))\
    | unlinked_lg_index.shift(1)).cumsum()
    
    distance_df.loc[distance_df[distance_df.linked == 1].index, 'linkage_group']  = tmp_linkage_groups
    
    distance_df = distance_df.loc[:,['contig', 'query', '3_target',\
                                            '3_distance', '5_target', '5_distance', 'linked', 'linkage_group']]
    #add a frequency columns to the dataframe
    distance_df['lg_freq'] = distance_df.groupby('linkage_group')['linkage_group'].transform('count')
    
    #now make a bin count dataframe
    
    #get the counts for each lg_freq == total number of genes in a bin of size lg_freq
    bins = distance_df[distance_df.linked !=0 ].groupby('lg_freq').count()
    
    #now get unlinked total number of genes ina bin size of 1
    bin_one = distance_df[distance_df.linked ==0 ].groupby('lg_freq').count().reset_index(drop= True)

    bin_one.index = [1] 
    #combine both
    all_bins = bins.append(bin_one)
    #use the index which represents the bin size
    all_bins['bin_size'] = all_bins.index
    
    all_bins = all_bins.sort_values('bin_size').reset_index(drop=True).loc[:, ['linked', 'bin_size']]

    all_bins.rename(columns={'linked': 'member_count'}, inplace=True)

    all_bins['number_of_bins'] = all_bins['member_count'] / all_bins['bin_size']

    #new_cnames = ['%s_%s' % (x,label) for x in all_bins.columns]

    #all_bins.rename(columns=dict(zip(all_bins.columns, new_cnames)), inplace=True)
    
    all_bins['label'] = label
    
    #all_bins['bin_size'] = all_bins['bin_size_' + label]
    return distance_df, all_bins

In [ ]:
max_distance = 12000

In [ ]:
distance_effector_df, bin_effector_df = _5_to_3_chains(e_to_e_sub_5,e_to_e_sub_3,max_distance=max_distance, label='effector') 

In [ ]:
distance_busco_df, bin_busco_df = _5_to_3_chains(p_busco_bed_rand_sub_5,p_busco_bed_rand_sub_3,max_distance=max_distance, label='busco')

In [ ]:
distance_all_gene_df, bin_all_gene_df = _5_to_3_chains(all_all_rand_5,all_all_rand_3,max_distance=max_distance, label = 'all_genes')

In [ ]:
overall_bining_df = pd.concat([bin_effector_df,bin_busco_df,bin_all_gene_df ])

In [ ]:
#set yourself up for the plots
overall_bining_df['member_count_log'] = np.log2(overall_bining_df.member_count)

conversion_dict = dict(zip(overall_bining_df.label.unique(), range(1,4)))
conversion_dict_color = dict(zip(overall_bining_df.label.unique(), ['r', 'g', 'b']))

#get the data labels and the color labels
overall_bining_df['number_labels'] = overall_bining_df.label.apply(lambda x: conversion_dict[x])
overall_bining_df['color_labels'] = overall_bining_df.label.apply(lambda x: conversion_dict_color[x])

#set the overall sns style
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})

#start the figure
fig, ax = plt.subplots(1,1)
title = "Linkage of genes with a maximum distance of %i" % max_distance
#fill the figure with a scatter plot
ax.scatter(overall_bining_df.bin_size, overall_bining_df.number_labels, s=overall_bining_df.member_count, \
           color =overall_bining_df['color_labels'] )
#add the labels
for label, x, y in zip(overall_bining_df.member_count,overall_bining_df.bin_size, overall_bining_df.number_labels ):
    plt.annotate(label, xy =(x+0.25,y))
plt.xlabel('bin size')
ax.set_yticks(range(1,4))
ax.set_yticklabels(overall_bining_df.label.unique())
plt.title(title)
out_file_name = "_".join(title.split(' '))
fig.savefig(os.path.join(OUT_FOLDER_FIG, out_file_name+'.png'), dpi=600)

Quick look at allele analysis

In [ ]:
allele_QC_fn = os.path.join('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis/alleles'\
                           , 'Pst_104E_v12_p_ctg.h_contig_overlap.Qcov80.PctID70.alleles')

allele_blast_df_fn = os.path.join('/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/allele_analysis' ,'Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastp.outfmt6.allele_analysis')

alleles_df = pd.read_csv(allele_QC_fn, header=None, sep = '\t', names=['p_protein', 'h_protein'])
alleles_df['match'] = alleles_df.p_protein + alleles_df.h_protein

allele_blast_df = pd.read_csv(allele_blast_df_fn, sep='\t')
allele_blast_df['match'] = allele_blast_df.Query + allele_blast_df.Target

allele_blast_df = allele_blast_df[(allele_blast_df.match.isin(alleles_df.match))]

len(allele_blast_df)

allele_blast_df.head()

allele_blast_df[(allele_blast_df.Query.isin(p_effector_list))]['PctID'].mean()

allele_blast_df[~(allele_blast_df.Query.isin(p_effector_list))]['PctID'].mean()

print(len(allele_blast_df[(allele_blast_df.Query.isin(p_effector_list))])/len(p_effector_list))

print(len(allele_blast_df)/len(p_allgene_bed))